In [2]:
addpath ("../files")
setenv('GNUTERM','qt')
format long

In [7]:
%%file ../files/metodoblinterpol2.m

function [W,x,iter] = metodoblinterpol2(fname, x, method='newton', line_search='quadratic',
                                        plot_iterations = true, tol=1.e-04, maxiter = 50, 
                                        maxjter = 300, c1 = 0.1)
%{
    Método de búsqueda de línea con la primer condición de Wolfe
    usando máximo descenso.
    usando los métodos de descenso:
        1. Máximo descenso
        2. Método de Newton
    Input:
        
        fname.- cadena de caracteres con el nombre de la función a minimizar.
        x    .- vector n-dimensional.
    
    Output:
        x    .- vector n-dimensional con la aproximación al mínimo local.
        iter .- contador con el número final de iteraciones externas.
%}
    
    iter = 0;        
    p = 1; s = -1;

    fx = feval(fname,x);
    g  = gradiente(fname,x);
    ng = norm(g);
    W  = [x]; 

    while ( ng > tol && iter < maxiter &&  norm(p)> 1.e-04)

        if (strcmp(method,'max'))
            p = -g;
        end

        if (strcmp(method,'newton'))
            H = hessian(fname,x);
            p = -H\g; 
        end

        %-----------------------------------------
        %            Búsqueda de línea
        
        alfa  = 1;              
        xt = x + alfa*p;        
        f  = feval(fname,x);  
        f1 = feval(fname, xt);
        s  = p'*g;            
        jter = 0;             

        if (abs(s) < 1.e-06)
            disp('No existe suficiente descenso  ')
            disp(sprintf('%2.0f  %2.10f',iter, s   ))
            iter = maxiter; 
        end

        while( (f1>f+alfa*c1*s) && jter < maxjter)  % búsqueda de línea
        
            if (strcmp(line_search,'linear'))
                alfa = alfa/2;
                xt = x + alfa*p;
                f1 = feval(fname, xt);
                jter = jter +1;
            end
            
            if (strcmp(line_search,'quadratic'))
                d2 = f1 - f - s;
                alfa = -( s)/(2*d2);       
                f1 = feval(fname, x + alfa*p);
                jter = jter +1;    
            end
        end   
        
        %          Fin de búsqueda de línea
        %------------------------------------------
        
        
        %------------------------------------------
        %                Graficación

        if (plot_iterations == true)
        
            t = linspace(0,1,50)';
            ft = zeros(50,1); rt = zeros(50,1);
            for k = 1:50        
                ft(k) = feval(fname, x+t(k)*p);  % función 
                rt(k) = fx + t(k)*(c1*p'*g);
            end

            fx = feval(fname,x+alfa*p);

            plot(t,ft,'--b',t,rt,'--m',alfa,fx,'dr', 'LineWidth',3)
            title('Gráfica de búsqueda de línea','Fontsize',16)
            xlabel('EJE  T','Fontsize',16)
            ylabel(' f(x + tp)','Fontsize',16  )
            legend('f(x)','recta','punto')
            pause
        end

        %              Fin de graficación
        %-----------------------------------------
          
        W = [W x];
        x = x + alfa*p;  
        fx = feval(fname,x); 
        g = gradiente(fname,x);
        iter = iter + 1;
        
    end      
end


Created file '/Users/sergio_arnaud/Documents/ITAM/8vo/applied_analysis/Labs/files/metodoblinterpol2.m'.


In [8]:
%plot gnuplot
[W,x,iter] = metodoblinterpol2('rosenbrock',[2.5,2.5]','newton','quadratic',false)

No existe suficiente descenso  
47  -0.0000000023
W =

 Columns 1 through 3:

   2.500000000000000e+00   2.500000000000000e+00   2.498005892035766e+00
   2.500000000000000e+00   2.500000000000000e+00   6.240025963241461e+00

 Columns 4 through 6:

   2.491079826503193e+00   2.483991287388984e+00   2.476731350184124e+00
   6.205423221723017e+00   6.170107187046128e+00   6.134040408048223e+00

 Columns 7 through 9:

   2.469291897024104e+00   2.461660429261820e+00   2.453824431572126e+00
   6.097190111682485e+00   6.059502559762726e+00   6.020924898006958e+00

 Columns 10 through 12:

   2.445773507611535e+00   2.437489901783958e+00   2.428959803591096e+00
   5.981415683696901e+00   5.940898416992898e+00   5.899317292626544e+00

 Columns 13 through 15:

   2.420163535687273e+00   2.411084451212724e+00   2.401696612924948e+00
   5.856589296124090e+00   5.812647845674472e+00   5.767383231033708e+00

 Columns 16 through 18:

   2.391976240865145e+00   2.381895345014450e+00   2.3714205471966

In [ ]:
%plot gnuplot
metodoblinterpol2('fcuad',[2.5,2.5]','newton','linear',true)